<center><b>Proposed changes as envisaged with respect to import functionality in moving towrds new design. </b></center>


<img src ="importchangeshld.png" />

The above design is as per the CQRS philosophy. Chnage is sought primarily with respect to the "C-Command" of the CQRS with respect to Add/Update/Delete operations to primarily enable a cleaner seaparation of the logic across different application layers/tiers. The "Q-Query" of the CQRS is best left to retain the existing logic of receiveing data from the BAService or SOADataGridService till the time the frontend application is not ready to be changed.

Detailed explanation

Tracing back to history Emisoft platform is based on a solution which is modelled according to an industry domain wherein the solution depicts the entities and the relationships as pervalent in the domain and this is primarily modelled by the configurators using the solution designer. When it comes to designing the runtime for the end users based on this solution during the software development stages effectively it tended to fit the domain driven design philosphy whereby if the domain entities with their respective behavior and validations self contained  could be generated based on how the solution configurators defined it then it should be very easy to design the application in true DDD philosphy. 
In the existing design the DDD tenet is partially done due to the true types of the entities not being run at dynamic and over the yers much thought has been gone to how to utilize the true types of the entities by dint of which the application could be made much simpler and also performance efficient. Also it needs to be borne in mind that during the initial conception stage the idea was way ahead of the technology but now new technology trends are emerging in the Microsoft space itself with .Net Core whereby doamin entiy types as in the true sense modelled by the configurators acn be utilized as is by the platform runtime.
Hereby we are presented a chnace to capitalize on the technology trend and move on to the .Net Core platform to lay the foundation for future complete migration by the overhaul required of impot functionality.

Technical details

The different players involved as is being conceptualized.


Domain entity(ies)
    

The generated server model containing the domain entities along with the different methods as designed by the solution configurators will be the domain entities coming into play. As we see withut much chnage it can be made comaptible to .Net standard and made use of in .Net Core as is. For transfer over the wire from UI to the web server whether DTO needs to be modelled on this domain entity is a personal choice and DTOs can be generated if deemed necessary. But since now over web api we use json to map an entity so we donot necessarily need DTO. AS we are not modifying the query operations so all the lquery logic as existiung written in the methods of this entities can be left as is.

Data web API

This is the proposed new web API that will expose methods primarily to conform to POT, PUT, DELETE verbs as prevalent in REST terminology and which will accept type in the form of json string conforming to domain entity. Any front end or relevant will communicate with this web api as far as CUD operations are concerned.

Domain Service layer (optional)

When using asp.net core and EF Core this layer is a bit meaningless as EF Core DbContext itself acts as the UOW-repository combination and if we have modelled the domain entity to be non-anaemic then the service layer becomes irrelevant. It thus becomes a personal choice to use this.

Domain validator

This can be used to ensure cleaner separation of the validation logic with respect to the domain entities as might be defined by the solution modelled from the main api layer.

Solution DBContext

The backbone when using EF Core. A custom DBContext inherited from the DBContext pertaining to the solution needs to be generated whereby for every domain entity there is a corresponding DBSet defined in this DBContext. This DBContext are what maps the domain entities to the corresponding underlying datbase tables and the dbcontext can also contain the rules related to database validation duplicating the domain validation if need be. The DBContext can define Save, Rollback, Commit methods which are appropriately called by the appropriate methods in the data layer.